In this notebook, I aim to roll through an analysis across a single patient which can easily be looped for multiple patients. To do so, we will use the functions that are written out more explicitly in the step-by-step notebooks. 

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress
import pandas as pd
from mne.preprocessing.bads import _find_outliers

In [3]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils

## Load, pre-process and re-reference the neural data

In [4]:
base_dir = '/sc/arion' # this is the root directory for most un-archived data and results  
subj_ids = ['MS007']
mne_dict = {f'{x}': [] for x in subj_ids}
for subj_id in ['MS007']: 
    # Set paths
    load_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/neural/Day1'
    elec_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/anat/'
    elec_files = glob(f'{elec_path}/*labels.csv')[0]
    save_path = f'{base_dir}/work/qasims01/MemoryBanditData/EMU/Subjects/{subj_id}'
    
    # Load electrode data (should already be manually localized!)
    elec_data = pd.read_csv(elec_files)

    # Sometimes there's extra columns with no entries: 
    elec_data = elec_data[elec_data.columns.drop(list(elec_data.filter(regex='Unnamed')))]

    # Load neural data
    mne_data = lfp_preprocess_utils.make_mne(load_path=load_path, 
                                             save_path=save_path, 
                                             elec_data=elec_data, 
                                             format='edf')
    
    # Re-reference neural data
    mne_data_reref = lfp_preprocess_utils.ref_mne(mne_data=mne_data, 
                                                  elec_data=elec_data, 
                                                  method='wm', 
                                                  site='MSSM')
    
    # Save this data so that you don't need this step again:
    mne_data_reref.save(f'{save_path}/wm_ref_ieeg.fif', overwrite=True)

    
    # Append to list 
    mne_dict[subj_id].append(mne_data_reref)
    

Extracting EDF parameters from /sc/arion/projects/guLab/Salman/EMU/MS007/neural/Day1/MS007_MemBandit.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1867007  =      0.000 ...  1823.249 secs...
Could not find a match for rhplt9.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6759 samples (6.601 sec)

Overwriting existing file.
Writing /sc/arion/work/qasims01/MemoryBanditData/EMU/Subjects/MS007/photodiode.fif
Closing /sc/arion/work/qasims01/MemoryBanditData/EMU/Subjects/MS007/photodiode.fif
[done]


/hpc/users/qasims01/resources/LFPAnalysis/LFPAnalysis/lfp_preprocess_utils.py:556: RuntimeWarning: This filename (/sc/arion/work/qasims01/MemoryBanditData/EMU/Subjects/MS007/photodiode.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  mne_data.save(f'{save_path}/photodiode.fif', picks='dc1', overwrite=True)


Could not find a match for rhplt9.
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=114, n_times=1867008
    Range : 0 ... 1867007 =      0.000 ...  1823.249 secs
Ready.
Added the following bipolar channels:
lacas1-lmolf1, lacas10-lacas9, lacas12-lacas9, lacas2-lmolf1, lacas3-lmolf1, lacas4-lacas8, lacas5-lacas8, lacas6-lacas8, lacas7-lacas8, laglt1-lhplt5, laglt10-laglt6, laglt2-lhplt5, laglt3-lhplt5, laglt7-lhplt6, laglt8-laglt6, laglt9-laglt6, laimm1-laglt5, laimm13-laimm12, laimm2-laimm6, laimm3-lmolf6, laimm4-laimm8, laimm5-laimm6, laimm7-laimm6, lcmfo1-lcmfo4, lcmfo12-lcmfo10, lcmfo13-lcmfo10, lcmfo2-lcmfo4, lcmfo3-lcmfo4, lcmfo7-lcmfo6, lcmfo8-lcmfo6, lhplt1-laglt5, lhplt10-lhplt8, lhplt2-laglt5, lhplt3-laglt4, lhplt4-lhplt6, lhplt9-lhplt8, lmcms1-lmcms5, lmcms2-lmcms5, lmcms3-lmcms5, lmcms4-lmcms5, lmcms9-lmcms8, lmolf2-lmolf6, lmolf3-lmolf6, lmolf4-laimm6, lmolf5-laimm6, lmolf8-laimm6, lmtpt1-lhplt5, lmtpt2-lhplt5, lmtpt3-lhplt5, lm

## Load the behavioral data

<RawEDF | MS007_MemBandit.edf, 114 x 1867008 (1823.2 s), ~1.59 GB, data loaded>